**Description:**

As a result of executing several optimization scenarios, we are going to get a bunch of data of all kinds. For instance, scenario configuration, scenarios names, and parameters and variables with different dimensions. All these considerations make it difficult to get a quick and clean data analysis.

In order to simplify this process, we have created two classes: SymbolsHandler, Symbol

**SymbolsHandler** help to load the symbol files created by using CollectScenariosPerSymbol class. The files are stored by default in the 'report_files' folder. Each file has an extension that represents the kind of results, either marginals (.m) or values (.v).

**Symbol** is an object that contains the data of a symbol but also enables mathematic operation between them by dealing with the dimensions and scenarios, as well as the scenario features.

In this notebook, we use Plotly express which matches up well with Symbol objects.

**Hint**: To run successfully this script make sure you are running this notebook on the root of your project folder where the manage.py is hosted.

In [ ]:
from dieterpy import SymbolsHandler, Symbol, storagecycling
import plotly.express as px

In [ ]:
SH = SymbolsHandler('folder')
SH

In [ ]:
# Get Z Symbol (Total system cost in DIETER)
Z = Symbol(name="Z", value_type="v", symbol_handler=SH)

In [ ]:
# see the data
Z.df

In [ ]:
Z.info

In [ ]:
Z.dims

In [ ]:
Z.items

In [ ]:
N_TECH = Symbol("N_TECH", "v", symbol_handler=SH)

In [ ]:
N_TECH.dfm

In [ ]:
N_TECH.info

In [ ]:
N_TECH.dims

In [ ]:
N_TECH.items

In [ ]:
con1a_bal = Symbol("con1a_bal", "m", symbol_handler=SH)

In [ ]:
con1a_bal

In [ ]:
con1a_bal.info

In [ ]:
# First operation: Symbol * number

In [ ]:
PRICE = con1a_bal * -1   # This operation creates a new Symbol

In [ ]:
PRICE                    # Look at the name = '(con1a_bal)*-1'

In [ ]:
PRICE.df

In [ ]:
# Second operation: multiplication of two symbols. First we get the hourly capacity factor of renewables as its availability.
phi_res = Symbol("phi_res", "v", symbol_handler=SH)
phi_res

In [ ]:
phi_res.info

In [ ]:
# Now the multiplication
TOTAL_vRES_GEN = N_TECH * phi_res

In [ ]:
TOTAL_vRES_GEN     # see symbol_type 'expression' as it is a Symbol created from other symbols, always "v" as value_type

In [ ]:
TOTAL_vRES_GEN.df

In [ ]:
TOTAL_vRES_GEN.items

In [ ]:
# As this is an 'expression' Symbol it has None info
TOTAL_vRES_GEN.info

In [ ]:
# We can add a desription (optional)
TOTAL_vRES_GEN.info = "Electricity generated by vRES: disptched + curtailed [MWh]"


In [ ]:
# Third operation: Addition of two symbols. First we get the symbols that will be added
G_L = Symbol("G_L", "v", symbol_handler=SH)
G_L

In [ ]:
G_L.info

In [ ]:
G_RES = Symbol("G_RES", "v", symbol_handler=SH)
G_RES

In [ ]:
G_RES.info       # Dispatched electricity

In [ ]:
# Now the addition
G_TECH = G_L + G_RES

In [ ]:
G_TECH

In [ ]:
G_TECH.items

In [ ]:
# Fourth operation: Subtraction. First we get the Symbol
d = Symbol("d", "v", symbol_handler=SH)
d

In [ ]:
d.info

In [ ]:
RESIDUAL_LOAD = d - TOTAL_vRES_GEN # This line will lead to an error. See comment bellow.

In [ ]:
# The error above says that the two Symbols differs in dimensions. 
# In this case, we could try to remove the 'tech' dimension from TOTAL_vRES_GEN by doing dimension reduction (default 'sum', it could be 'mean', 'max', and more)
RESIDUAL_LOAD = d - TOTAL_vRES_GEN.dimreduc('tech')

In [ ]:
RESIDUAL_LOAD

In [ ]:
fig = px.line(data_frame=RESIDUAL_LOAD.df,
        x='h',
        y='value',
        facet_row='n',
        color='id',
        hover_name='id',
       )
fig.show()

In [ ]:
# In case you want to generate the residual load duration curve uncomment this snippet

# import pandas as pd
# lt = []
# for ix, gr in RESIDUAL_LOAD.df.sort_values(by='value', ascending=False).groupby(["id", "n"]):
#     dt = gr.copy()
#     dt.loc[:, "hr"] = [i for i in range(1, len(gr) + 1)]
#     lt.append(dt)
# data = pd.concat(lt)

# fig = px.line(data_frame=data,
#         x='hr',
#         y='value',
#         facet_row='n',
#         color='id',
#         hover_name='id',
#        )
# fig.show()

In [ ]:
# Fifth operation: Division of symbols
COST_MWh = Z/G_TECH.dimreduc('tech').dimreduc('h').dimreduc('n')

In [ ]:
COST_MWh

In [ ]:
COST_MWh.df

In [ ]:
COST_MWh.info = "Total system cost per total generation [€/MWh]"

In [ ]:
G_TECH.items

In [ ]:
# Add dimension
G_TECH_type = G_TECH.add_dim(dim_name='type', value={'tech': {'CCGT': 'Conventional',
                                                              'OCGT': 'Conventional',
                                                              'bio': 'Renewables',
                                                              'hc': 'Conventional',
                                                              'lig': 'Conventional',
                                                              'nuc': 'Conventional',
                                                              'oil': 'Conventional', 
                                                              'other': 'Conventional',
                                                              'pv': 'Renewables',
                                                              'ror': 'Renewables',
                                                              'wind_off': 'Renewables',
                                                              'wind_on': 'Renewables'
                                                              }
                                                    }
                                )

In [ ]:
G_TECH_type

In [ ]:
GEN_SHARE = G_TECH_type.dimreduc('tech').dimreduc('h')/G_TECH.dimreduc('tech').dimreduc('h')

In [ ]:
GEN_SHARE

In [ ]:
GEN_SHARE.df

In [ ]:
# Finally some charts

In [ ]:
N_STO_E = Symbol("N_STO_E", "v", symbol_handler=SH)

In [ ]:
N_STO_E.df

In [ ]:
N_STO_E.dfm

In [ ]:
N_STO_E.dfm_nan   # For some charts this method .dfm_nan becomes useful.
                  # Features elements of scenarios that have None values (NaN) are replaced by -1. Only for numeric features.
                  # From Above dataframe: a numeric feature is 'phi_min_res(n)' and its elements are 0.8 and 0.9.
                  # in this example there are no difference between .dfm and .dfm_nan

In [ ]:
fig = px.bar(data_frame=(N_STO_E/1000000).dfm,    # 1000000 is the conversion factor from MWh to TWh
        x='id',
        y='value',
        color='sto',
        facet_row='n',
        facet_col='phi_min_res(n)',
        labels = {'value': 'Storage energy capacity [TWh]'},
       )
fig.show()

In [ ]:
# Code Comparison

# GAMS: 
# sum(h2_channel,h2_aux_pretrans_sw(n,h2_channel) * h2_aux_pretrans_ed(n,h2_channel) * (sum(h2_tech,h2_bypass_1_sw(n,h2_tech,h2_channel) * H2_BYPASS_1.l(n,h2_tech,h2_channel,h)) + H2_STO_P_OUT.l(n,h2_channel,h)))

# Symbol class:
# (h2_aux_pretrans_sw * h2_aux_pretrans_ed*((h2_bypass_1_sw * H2_BYPASS_1).dimreduc('h2_tech')) + H2_STO_P_OUT).dimreduc('h2_channel')

In [ ]:
# Finally. Check if some of your storage technologies in any scenario is affected by storage cycling
# First, load variables for hourly input and output storage

STO_IN = Symbol("STO_IN", "v", symbol_handler=SH)
STO_OUT = Symbol("STO_OUT", "v", symbol_handler=SH)

In [ ]:
# use the function storagecycling it returns a Symbol with value 1 at the time of charging or discharging and 0 if nothing occurs
STO_Bin = storagecycling(STO_IN, STO_OUT)  # if 2 is present at any hour, it means that discharge and charge occurred 

In [ ]:
STO_Bin.df

In [ ]:
STO_Bin.df.query("value == 2")

In [ ]:
# The last chart. Visualize STO_IN and STO_OUT together in one symbol
STO_inout = STO_IN.add_dim('flow','charge') + STO_OUT.add_dim('flow','discharge') # Hint: Avoid using existing dimension names from other symbols.

In [ ]:
STO_inout.df

In [ ]:
fig = px.line(data_frame=STO_inout.df,
        x='h',
        y='value',
        color='flow',
        line_dash = 'sto',
        facet_row='id',
        facet_col='n',
        labels = {'value': '(Dis)charging [MWh]'},
       )
fig.show()